## <b>This notebook merges the crime and census data</b>
The merged datasets( consolidated crime and census datasets) are loaded into the datawarehouse ( AWS RDS Postgres )
***

### <b>Modules Required</b>

In [2]:
%load_ext autoreload
%autoreload 2
from etl_helper import * 


import pandas as pd
import glob
import os 
import numpy as np
import re

import configparser
from etl_helper import * 


 
import configparser

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### <b>POA</b>

#### Combine the merged (census merged ) and the crime dataset 

In [ ]:
crim_categories = GetData('''SELECT "Subcategory" FROM prod."bocsar.crimestats.rate" GROUP BY "Subcategory";''')


In [ ]:
crim_categories

In [ ]:
gcp_Data_from_RDS = GetData('''SELECT * FROM stage."2021_census.gcp_all.poa";''')


In [ ]:
gcp_Data_from_RDS.T


In [ ]:
#dfs = crim_data_with_rate.groupby('Subcategory')
gcp_all = pd.read_csv('/Users/icmec/Documents/project_census_crime/data/2021_census.gcp_all.poa.csv')

gcp_all['poa_code_2021'] =  gcp_all['poa_code_2021'].str.extract('(\d+)')

gcp_all = gcp_all.astype( { 'poa_code_2021':'int64' }  )

crim_data_with_rate = GetData('''SELECT * FROM prod."bocsar.crimestats.rate";''')

for name, group in crim_data_with_rate.groupby('Subcategory'):
    gcp_all = gcp_all.merge( group[['Postcode','TotalCrimes','rate']].rename( columns = { 'Postcode':'poa_code_2021' }), left_on = "poa_code_2021", right_on = "poa_code_2021", how = "left")
    name = re.sub(r'[^\w\s]','',name)
    camelCaseName = name.replace(' ','_').lower()
    gcp_all.rename( columns= {'TotalCrimes' : f'{camelCaseName}.TotalCrimes' , 'rate':f'{camelCaseName}.rate' } , inplace=True )

In [ ]:
currentDirectory = os.getcwd()
gcp_all.to_csv( f'{currentDirectory}/data/2021_census.gcp_all_with_offences.poa.csv', index=False )

### <b>Merging  LGA data AND Juvenile CRIME</b>  

In [ ]:
lga_codes_and_names = pd.read_csv(f'{currentDirectory}/data/georef-australia-local-government-area.csv' , usecols=['Official Code Local Government Area','Official Name Local Government Area'],delimiter=';')


In [ ]:
lga_codes_and_names.set_index('Official Name Local Government Area', inplace=True)


In [ ]:
lga_codes_and_names.rename( columns = {'Official Code Local Government Area':'lga_code'} , inplace = True )
lga_codes_and_names.index.name = 'lga_name'

In [ ]:
lga_codes_and_names


In [ ]:
juvenile_crime_data = pd.read_excel(f'{currentDirectory}/data/juv_victims_by_lga_gender_age.xlsx')


In [ ]:
juvenile_crime_data


In [ ]:
merged_code_name_ = juvenile_crime_data.merge( lga_codes_and_names, how='left' , left_on= 'lga' , right_on='Official Name Local Government Area' , ) 


In [ ]:
merged_code_name = merged_code_name[ ['Homicide', 'Domestic violence related assault',
       'Non-domestic violence related assault', 'Sexual offences',
       'Abduction and kidnapping', 'Robbery', 'Blackmail and extortion',
       'Intimidation, stalking and harassment',
       'Other offences against the person', 'Arson',
       'Breach Apprehended Violence Order',
       'Official Code Local Government Area']  ]

In [ ]:
# Cootamundra-Gandagai == 12160
# Hunters Hill == 14100
# Unincorporated NSW = 19399
# { 19499, 19799}

# 14170

#merged_code_name.drop([56,73,86,114] , axis = 0 , inplace = True )

#a = list(set( juvenile_crime_data['lga']  ) - set(lga_codes_and_names['Official Name Local Government Area']))

#merged_code_name['Official Code Local Government Area'] =  merged_code_name['Official Code Local Government Area'].astype(int)

In [ ]:
SendData( merged_code_name, table_name='2021.juvenile_crime_data.stats',index=True,schema_name='stage' , if_exists_flag='replace' )


#### Joining Juvenile crime AND consolidated lga census data

In [ ]:
merged_code_name.set_index('Official Code Local Government Area', inplace = True)


In [ ]:
def getCamelCaseColNames(list_of_columns):
    camelCaseName = []
    for  name in list_of_columns:
        names = re.sub(r'[^\w\s]','',name)
        camelCaseName.append(f"{names.replace(' ','_').lower()}.count")
    return camelCaseName

In [ ]:
merged_code_name.columns  =  camelCaseName


In [ ]:
#merge
consolidated_lga_census_crime = lga_census.merge( merged_code_name, how='inner' , left_on='lga_code_2021', right_index=True )

In [ ]:
for each in merged_code_name.columns:
    name, _ = each.split('.')
    consolidated_lga_census_crime[ f'{name}.rate' ] =  consolidated_lga_census_crime[each] / consolidated_lga_census_crime['g01.tot_p_p']

In [ ]:
df = pd.read_csv(f'{currentDirectory}/data/2021_census.gcp_all_juvenile_victims.lga.csv')


In [ ]:
lga_codes_and_names.reset_index(inplace=True)


In [ ]:
lga_codes_and_names


In [ ]:
df = df.merge(lga_codes_and_names, how='inner' , left_on='lga_code_2021' , right_on='lga_code'  )
df.drop(['lga_code'], inplace =True, axis = 1)
df.set_index('lga_name',inplace=True)
df.index.rename = '2021.lga_name'


df_copy = df.copy( deep=True)
df_copy = df_copy.replace('..',np.nan)
df_copy.infer_objects().dtypes
df_copy.T


SendData(df_copy.T,table_name='2021_census.gcp_all_juvenile_victims.lga', schema_name = 'prod' , if_exists_flag = 'replace' , index=True )
GetData( '''SELECT * FROM prod."2021_census.gcp_all_juvenile_victims.lga" LIMIT 10'''  ).T

consolidated_lga_census_crime.to_csv(f'{currentDirectory}/data/2021_census.gcp_all_juvenile_victims.lga.csv', index=False)
SendData(consolidated_lga_census_crime.T,table_name='2021_census.gcp_all_juvenile_victims.lga', schema_name = 'prod' , if_exists_flag = 'replace' , index=True )

### <b>Merging  LGA data AND Offenders</b> 

In [ ]:
lga_codes_and_names = pd.read_csv(f'{currentDirectory}/data/lga/lga_code_and_names.csv')
lga_offenders_10_17 = pd.read_csv(f'{currentDirectory}/data/lga/lga.offenders_10_17.csv')

print(f'lga_codes_and_names = {lga_codes_and_names.shape}' )
print(f'lga_offenders_10_17 = {lga_offenders_10_17.shape}' )

display(lga_codes_and_names.head(5))
display(lga_offenders_10_17.head(5))

In [ ]:
lgaJuvenileOffenders_lgaName = lga_offenders_10_17.merge( lga_codes_and_names , left_on='lga' , right_on='lga', how = 'inner' )


In [ ]:
lgaJuvenileOffenders_lgaName.set_index('lga', inplace = True)
lgaJuvenileOffenders_lgaName

In [ ]:
def getCamelCaseColNames(list_of_columns):
    camelCaseName = []
    for  name in list_of_columns:
        names = re.sub(r'[^\w\s]','',name)
        camelCaseName.append(f"{names.replace(' ','_').lower()}.count")
    return camelCaseName

camelCaseColNames = getCamelCaseColNames(lgaJuvenileOffenders_lgaName.columns)

In [ ]:
lgaJuvenileOffenders_lgaName.columns = camelCaseColNames


In [ ]:
lgaJuvenileOffenders_lgaName.rename(columns= {'lga_code_2021.count':'lga_code_2021'}, inplace=True)


In [ ]:
lga_census['lga_code_2021'] =  lga_census['lga_code_2021'].str.extract('(\d+)')

lga_census['lga_code_2021'] = lga_census['lga_code_2021'].astype(int)


In [ ]:
lgaCensus_juvenileOffenders =  lga_census.merge(lgaJuvenileOffenders_lgaName,left_on='lga_code_2021',right_on='lga_code_2021',how='inner')

#======================================================================================================================================


for each in lgaJuvenileOffenders_lgaName.columns[1:-1]:
    name, _ = each.split('.')
    lgaCensus_juvenileOffenders[ f'{name}.rate' ] =  lgaCensus_juvenileOffenders[each] / lgaCensus_juvenileOffenders['g01.tot_p_p']

#======================================================================================================================================

lgaCensus_juvenileOffenders.set_index('lga_name',inplace = True)
lgaCensus_juvenileOffenders.replace('..',np.nan, inplace=True)


#======================================================================================================================================

SendData(lgaCensus_juvenileOffenders.T,table_name='2021_census.gcp_all_juvenile_offenders.lga', schema_name = 'prod' , if_exists_flag = 'replace' , index=True )

In [ ]:
## Save the LGA offenders  data from postgres to the archive folder 
gcp_all_juvenile_offenders = GetData(sql='SELECT * FROM prod."2021_census.gcp_all_juvenile_offenders.lga"')
gcp_all_juvenile_offenders = gcp_all_juvenile_offenders.T
gcp_all_juvenile_offenders = gcp_all_juvenile_offenders.rename(columns=gcp_all_juvenile_offenders.iloc[0]).drop(gcp_all_juvenile_offenders.index[0])

In [ ]:
currentDirectory = os.getcwd()
gcp_all_juvenile_offenders.to_csv(f'{currentDirectory}/data/lga/2021_census.gcp_all_juvenile_offenders.lga.csv',index=True)

## MERGING THE CENSUS 2016 AND CRIME 2016 DATA. 

In [ ]:
lga_census_2016 = pd.read_csv('/Users/icmec/Documents/project_census_crime/data/lga/2016_census.gcp_all.lga.csv' , index_col=0 )
lga_codes_names = pd.read_csv('/Users/icmec/Documents/project_census_crime/data/lga/lga_code_and_names.csv')
lga_census_2021 = pd.read_csv('/Users/icmec/Documents/project_census_crime/data/lga/2021_census.gcp_all_juvenile_victims_with_names.lga.csv')

crime_data_2016 = pd.read_csv('/Users/icmec/Documents/project_census_crime/data/lga/2016_crime_data.csv')

In [ ]:
lga_census_2016['lga_code_2016'] =  lga_census_2016['lga_code_2016'].str.extract('(\d+)')
lga_census_2016['lga_code_2016'] = lga_census_2016['lga_code_2016'].astype(int)

In [ ]:
#merging to get LGA names 
lga_census_2016_with_names =  lga_codes_names.merge(lga_census_2016 , left_on='lga_code_2021',right_on='lga_code_2016',how='inner'  )

In [ ]:
#merging the LGA census and crime data
lga_census_2016_with_names.drop('lga_code_2021',axis=1,inplace=True)
lga_census_2016_with_names_crime = lga_census_2016_with_names.merge(crime_data_2016,left_on='Unnamed: 0',right_on='LGA',how='inner')
lga_census_2016_with_names_crime

In [ ]:
#computing the rate of crime 
for each in crime_data_2016.columns[1:]:
    name, _ = each.split('.')
    print(name)
    lga_census_2016_with_names_crime[ f'{name}.rate' ] =  lga_census_2016_with_names_crime[each] / lga_census_2016_with_names_crime['g01.tot_p_p']

In [ ]:
lga_census_2016_with_names_crime.rename( columns = { 'Unnamed: 0':'lga_name' } , inplace = True )
lga_census_2016_with_names_crime
lga_census_2016_with_names_crime.to_csv( '/Users/icmec/Documents/project_census_crime/data/lga/2016_census.gcp_all_juvenile_victims_with_names.lga.csv' )

In [ ]:
 
lga_census_2016_with_names_crime.drop('LGA' , axis=1 , inplace= True)
transformed_ = lga_census_2016_with_names_crime.T
transformed_.columns = transformed_.iloc[0]
transformed_1 =  transformed_.iloc[1:].replace('..',np.nan)
transformed_1.astype('float')
transformed_1.reset_index()
SendData( transformed_1 , table_name= '2016_census.gcp_all_juvenile_victims.lga' ,schema_name='prod',if_exists_flag='replace' , index=True )
# SendData( transformed_.iloc[1:] , table_name= '2016_census.gcp_all_juvenile_victims.lga' ,schema_name='prod',if_exists_flag='replace' )

## Merging the QLD Census and Crime

In [ ]:
#nameLgaQld = pd.read_csv('/Users/icmec/Documents/project_census_crime/data/crime/georef-australia-local-government-area.csv' , delimiter=';' , usecols = ["Official Code Local Government Area","Official Name Local Government Area","Long Official Name Local Government Area"] , index_col=0 )
#nameLgaQld.index.name = 'lga_code_2021'
#nameLgaQld.sort_values('Official Name Local Government Area')
nameLgaQld.sort_values('Official Name Local Government Area').to_csv('/Users/icmec/Documents/project_census_crime/data/crime/nameLgaQld.csv', index=True)

In [20]:
qld_census = pd.read_csv('https://data-lake-icmec-au.s3.ap-southeast-2.amazonaws.com/projectCrimeCensus/landingZone/2021_census.gcp_all.lga_qld.csv' )
lgaNameQld = pd.read_csv('/Users/icmec/Documents/project_census_crime/data/crime/nameLgaQld.csv')
VictimsCrimeQld = pd.read_csv('/Users/icmec/Documents/project_census_crime/data/crime/victimeCrimeQld.csv')


In [35]:
VictimsCrimeQld.T.to_csv('/Users/icmec/Documents/project_census_crime/data/crime/transformedVictimeCrimeQld.csv',header=False)
#pd.DataFrame(VictimsCrimeQld.columns ,  columns= ['Crimes'] , index=None ).sort_values('Crimes').to_clipboard()

#where we left of is at name   mapping the crime of QLD to NSW data